# Семинар 7: продвинутые функции

### Рекурсия

Пусть мы хотим написать функцию быстрого возведения числа в степень:

```
def pow(base: Union[int, float], deg: int)
```

Известно, что:
```
pow(a, n)
    = pow(a, n // 2) ** 2, если n -- четное
    = a * pow(a, n // 2) ** 2, если n -- нечетное
```

In [ ]:
from typing import Union


def pow(base: Union[int, float], deg: int):
    if base == 0:
        return 0  # а без этого получим DivisionByZeroError, если deg < 0 [1]

    if deg == 0:
        return 1  # без этого получим бесконечное выполнение
    elif deg < 0:
        deg *= -1
        base = 1 / base  # [1] вот тут

    if not deg % 2:
        return pow(base, deg // 2) ** 2
    return base * pow(base, deg // 2) ** 2


In [ ]:
pow(2, 2)

Что тут произошло? Правильно, мы вызвали из функции саму себя, получается так называемая рекурсия.

С рекурсией стоит быть аккуратным: неаккуратное ее использование приведет к бесконечной работе программы, а также она выжрет у вас всю память компа или питон прибьет ее раньше.

Под капотом рекурсия работает как стэк вызовов. Каждый новый вызов помещается на его верхушку, после выполнения работы функции на этапе рекурсии этот вызов со стэка снимается. Проще всего это заметить если Сделать искусственный пример, где выдадим ошибку в какой-то момент времени.

In [ ]:
def recursive(n: int):
    if n == 0:
        raise RuntimeError("Just checking call stack")

    recursive(n - 1)


In [ ]:
recursive(6)

### Проблема лимита рекурсии

Пусть мы хотим рекурсивно сумму для всех чисел от 1 до n (примечание: ДА, я знаю, что это можно просто сделать циклом, это игрушечный пример)

In [ ]:
def recursive(n: int):
    if n == 1:
        return 1

    return n + recursive(n - 1)

In [ ]:
recursive(3)

In [ ]:
recursive(10000)  # ой, а че это мы такое поймали?

In [ ]:
import sys

sys.getrecursionlimit()  # по умолчанию в питоне очень маленький лимит на рекурсивные вызовы

In [ ]:
sys.setrecursionlimit(10 ** 6)  # надо быть аккуратным

recursive(10000)

### Проблема отсутствия мемоизации

Пусть мы хотим посчитать $n$-ое число Фибоначчи

$F_1 = F_2 = 1$

$F_n = F_{n - 1} + F_{n - 2}$

In [ ]:
def fib(n: int):
    if n <= 2:
        return 1
    return fib(n - 1) + fib(n - 2)  # рекурсивно берем предыдущие числа Фибоначчи

А теперь давайте подумаем: сколько будет работать такой код?

In [ ]:
%%time
fib(8)

In [ ]:
%%time
fib(34)  # тут уже будет долго

In [ ]:
%%time
fib(40)  # и чем дальше, тем хуже, а 40 -- это же еще не то чтобы много

Проблема заключается в том, что у нас есть дубликаты вызовов. Поэтому питон делает много дополнительных действий. Чем дальше по номеру число Фибоначчи -- тем больше придется решать одинаковых абсолютно вызовов рекурсии. Решить это можно мемоизацией:

In [ ]:
from collections import defaultdict


SENTINEL = object()


fib_cache = defaultdict(lambda: SENTINEL)


def set_cache(n: int, result: int):
    fib_cache[n] = result


def fib(n: int):
    if fib_cache[n] is not SENTINEL:
        return fib_cache[n]

    if n <= 2:
        set_cache(n, 1)
        return 1
    
    result = fib(n - 1) + fib(n - 2)
    set_cache(n, result)
    return result

In [ ]:
%%time
fib(1000)  # быстро? быстро!

Очевидно, что это уже получается громоздко и неудобно, поэтому кэширование в питоне есть в том числе и встроенное.

Есть так называемые декораторы `@cache` и `@lru_cache` (least recently used). Второй выкидывает из кэша элементы, когда его размер достиг заданного максимума. При этом `@cache` устроен вот так:

```(python)
def cache(user_function, /):
    return lru_cache(maxsize=None)(user_function)
```

У `@lru_cache` параметр maxsize по умолчанию равен 128, можно увеличить при желании.

In [ ]:
from functools import cache, lru_cache

# lru_cache -- можно еще указать параметр maxsize
@lru_cache  # <--- это декоратор, про них поговорим чуть позже
def fib(n: int):
    if n <= 2:
        return 1

    return fib(n - 1) + fib(n - 2)

In [ ]:
%%time
fib(1000)

### Что еще умеет functools?

Есть какие-то очень специфичные функции, например, [cmp_to_key](https://docs.python.org/3/library/functools.html#functools.cmp_to_key), который не очень рекомендуется использовать без особой надобности.

Но есть и полезные штуки:

In [ ]:
from functools import partial

my_list = list(map(int, input().split()))

get_sorted_list = partial(sorted, my_list)  # создаем функцию из функции прямо в коде = sorted(my_list)
get_reverse_sorted_list = partial(sorted, my_list, reverse=True)  # = sorted(my_list, reversed=True)

In [ ]:
get_sorted_list()

In [ ]:
get_reverse_sorted_list()

Можно заранее и не передавать объект:

In [ ]:
from typing import Callable


def reverse_number(n: int) -> int:
    return int(str(n)[::-1])  # 456 -> 654

def sort_digits(n: int) -> int:
    return int("".join(sorted(str(n))))  # 672 -> 267

def add_one(n: int) -> int:
    return n + 1

def combine(*functions: Callable[[int], int], number: int) -> int:
    result = number
    for func in functions:
        result = func(result)
    return result

# по сути partial позволяет заполнить какие-то аргументы заранее, а остальные дозаполнить потом
do_some_stuff = partial(combine, add_one, reverse_number, add_one, add_one, sort_digits)
do_another_stuff = partial(combine, reverse_number, add_one, add_one, add_one)

In [ ]:
do_some_stuff(number=456)

In [ ]:
do_another_stuff(number=456)

#### Факультативное: singledispatch как начало полиморфизма

In [ ]:
from functools import singledispatch
from typing import List


@singledispatch
def dummy_sum(arg):
    print(f"I got unknown arg {arg}")
    return None


@dummy_sum.register
def _(arg: int):
    return arg  # непонятно как инты суммировать ага


@dummy_sum.register
def _(arg: list):
    return sum(arg)

In [ ]:
print(dummy_sum(5))

In [ ]:
print(dummy_sum([1, 2, 3]))

In [ ]:
print(dummy_sum("abc"))

# Практика

### Задача 1
Пусть у нас есть словарь любой вложенности. При этом на самом нижнем уровне у него всегда находится строка. Ключи тоже всегда строки. Надо написать функцию, которая принимает на себя такой словарь, и выводит на экран все строки на нижнем уровне, в любом порядке.

```(python)
a = {
    "x": {
        "y": "you_killed_kenny",
        "w": "south_park",
    },
    "p": "matt_stone",
}

print(recursive_print(a))
```

Для того, чтобы понять, что перед нами -- словарь или список -- можно воспользоваться функцией `isinstance(some_variable, dict)` -- она вернет True, если это словарь и False иначе

In [ ]:
a = {
    "x": {
        "y": "you_killed_kenny",
        "w": "south_park",
    },
    "p": "matt_stone",
}

recursive_print(a)

### Задача 2

Напишите программу, которая выбирает их полученной на вход последовательности (каждое число на новой строке) квадраты целых чисел выводит их в обратном порядке. Использовать массив для хранения последовательности не разрешается.

### Задача 3

Для быстрого вычисления наибольшего общего делителя двух чисел используют алгоритм Евклида. Он построен на следующем соотношении: $gcd(a, b) = gcd(b, a \mod b)$

Реализуйте рекурсивный алгоритм Евклида в виде функции gcd(a, b).